1. 텐서플로 소개
2. 텐서, 연신, 변수, 특별한 데이터 구조 
3. tf.keras에 있는 거의 모든 구성 요소를 커스터마이징
4. 텐서플로 함수가 어떻게 성능을 향상하는지 알아보기
5. **오토그래프와 트레이싱을 사용해 그래프를 생성하는 방법과 텐서플로 함수를 사용할 때 따라야 할 규칙**

### 텐서플로 함수

In [3]:
import tensorflow as tf

In [1]:
def cube(x):
    return x ** 3

In [2]:
cube(2)

8

In [4]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- `tf.function()` 을 통해 **파이썬 함수를 텐서플로 함수**로 바꿈

In [5]:
tf_cube = tf.function(cube)
tf_cube

In [6]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [7]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- 내부적으로 `tf.function()`은 `cube()`함수에서 수행되는 계산을 분석하고 동일한 작업을 수행하는 계산 그래프릉 생성한다.

#### `tf.function` 데코레이터
이는 텐서플로 함수를 생성하는 또다른 방법으로 전자의 방법보다 널리 사용됨.

In [8]:
@tf.function
def tf_cube(x):
    return x**3

- 원본 파이썬 함수는 필요할 때 여젼히 텐서플로 함수의 `python_function`속성으로 참조 가능

In [10]:
tf_cube.python_function(2)

8

- 기본적으로 텐서플로 함수는 호출에 사용되는 입력 크기와 데이터 타입에 맞춰 매번 1새로운 그래프를 생성한다. -> 다형성을 처리함
- 텐서플로는 사용하지 않는 노드를 제거하고 표현을 단순화하는 방식으로 계산 그래프를 최적화 한다 -> 효율적인 실행으로 원본 파이썬 함수보다 빠르게 실행됨.
- 사용자 정의 함수를 작성하고 케라스 모델에 사용할 때는 tf.function 필요 없음

### 오토그래프와 트레이싱
- 오토그래프 : 파이썬 함수의 소스 코드를 분석하여 for문, while문, if문은 물론 break, continue, return와 같은 제어문을 모두 찾는 단계
- 소스 코드를 분석하는 이유는 파이썬이 제어문을 찾을 수 있는 방법을 제공하지 않기 때문에
- 코드를 분석한 후 오토그래프는 **이 함수의 모든 제어문을 텐서플로 연산으로 바꾼 업그레이드된 버전**을 만든다.
- 트레이싱 과정을 통해 최종그래프 생성
- 노드(연산) 화살표 (텐서)
- 함수는 심볼릭 텐서를 매개변수로 하여 호출됨

###  텐서플로 함수 사용 방법
1. 넘파이나 표준 라이브러리를 포함해서 다른 라이브러리를 호출하면 트레이싱 과정에서 실행된다. 이 호출은 그래프에 포함되지 않으므로 트레이싱 과정에서 코드가 실행되는 것을 원치 않을 시 `np.sum()`대신 `tf.reduce_sum()`을, `sorted()`대신 `tf.sort()`를 사용하면 된다.   
    - 텐서플로가 지원하지 않는 코드가 부수적인 작업(pyton counter를 업데이트, 혹은 로깅)을 하면 함수를 트레이싱할 때만 호출되므로 텐서플로 함수를 호출할 때 이 코드가 실행되지 않는다.
    - 임의의 코드 `tf.py_function()`을 감쌀 수 있지만 최적화도 수행할 수 없기에 성능이 저하 된다. 또한 파이썬이 가능한 플랫폼에서만 이 그래프가 실행되므로 이식성 또한 낮다.
    
2. 다른 파이썬 함수나 텐서플로 함수를 호출할 수 있다. 데코레이터 적용할 필요가 없으며 이 함수들의 연산을 감지하기에 동일한 규칙이 따름.    
3. 변수를 생성할 때 처음 호출될 때만 수행되어야 한다.
    - 텐서플로 함수 밖에서 변수를 생성하는 것이 좋다
    - 변수에 새로운 값을 할당하려면 `=` 연산자 대신 `assign()` 메서드를 사용해야 한다. 
4. 파이썬 함수의 소스코드는 텐서플로에서 사용 가능해야 한다.
5. 텐서플로는 텐서나 데이터셋을 순회하는 for문만 감지한다.
    - `for i in range(x)` 대신 `for i intf.range(x)`를 사용해야 한다. 
6. 성능면에서는 반복문보다 가능한 한 벡터화된 구현을 사용하는 것이 좋다.

#### TF 함수가 계산 그래프를 추출하기 위해 파이썬 함수를 트레이싱하는 방법
https://github.com/SwimmingHwang/handson-ml2/blob/master/12_custom_models_and_training_with_tensorflow.ipynb